In [1]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 

# ZARA Scraping

In [2]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
# Slowly scrolling down the page
def scroll_down_page(driver, speed=1000):
    current_scroll_position, new_height= 0, 1
    while (current_scroll_position <= new_height):
        try:
            print(current_scroll_position)
            print(new_height)
            current_scroll_position += speed
            driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = driver.execute_script("return document.body.scrollHeight")
            time.sleep(5)
        except:
            PrintException()
            break

In [4]:
# Scraping the page
def scrape_detail_page(url,category,collection):
    
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.zara
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    check = driver.find_elements_by_xpath('/html/body/div[2]/section/div/section')
    path = check[len(check)-1].find_elements_by_xpath('div/ul/li')
    result = pd.DataFrame()
    for x in range(len(path)):
        element = path[x]
        element.location_once_scrolled_into_view
        try:
            # label = path[x].find_elements_by_xpath('div/div[1]/div')[0].text
            price = re.sub("\,","",re.sub(" IDR","",path[x].find_elements_by_css_selector('.price._product-price')[0].text))
            name = path[x].find_elements_by_css_selector('.name._item')[0].text
            sku = path[x].get_attribute('data-productid')
            image = path[x].find_elements_by_css_selector('img')[0].get_attribute('src')
            while(image == 'https://static.zara.net/stdstatic/1.108.1-b.2/images/transparent-background.png'):
                image = path[x].find_elements_by_css_selector('img')[0].get_attribute('src')
                time.sleep(2)
            # Return result
            date = datetime.datetime.now().strftime("%Y-%m-%d")
            new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':price, 'discounted_price':price, 'image':image,'category':category,'scraping_date':date}])
            result = result.append(new_result)
            
            # save to mongodb
            collection.insert_many(new_result.to_dict('records'))
        except:
            #PrintException()
            pass
    driver.close()
    return result

In [6]:
# MAIN
zara_category = pd.read_csv('C:\\Users\\tamu\\SharePoint\\Product Development - Documents\\Merchandising Analytics\\Analysis - QBR Jul19\\Data\\scraping\\zara_category.csv')
result = pd.DataFrame()
for x in range(0,len(zara_category)):
    category = zara_category.iloc[x]
    print(category['category'])
    result = scrape_detail_page(category['url'],category['category'],category['collection'])
    print(result.head())

blazers
Connected successfully!!!
  category discounted_price  \
0  blazers           699900   
0  blazers           699900   
0  blazers          1199900   
0  blazers          1199900   
0  blazers          1199900   

                                               image  \
0  https://static.zara.net/stdstatic/1.110.0-b.83...   
0  https://static.zara.net/photos///2019/I/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   

                           name origin_price scraping_date       sku  
0                  BASIC BLAZER       699900    2019-06-10  14172824  
0                  BASIC BLAZER       699900    2019-06-10  15554085  
0  CONTRAST SHAWL COLLAR BLAZER      1199900    2019-06-10  20800662  
0  CONTRAST SHAWL COLLAR BLAZER      1199900    2019-06-10  20799659  
0  CONTRAST SHAWL COLLAR BLAZER      1199900    2019-06-10  13238075  
jackets
Connecte

         category discounted_price  \
0  dresses > midi           699900   
0  dresses > midi           699900   
0  dresses > midi           999900   
0  dresses > midi           699900   
0  dresses > midi           699900   

                                               image  \
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   

                             name origin_price scraping_date       sku  
0                  BUTTONED DRESS       699900    2019-06-10  20522315  
0  STRAPPY DRESS WITH RUFFLE TRIM       699900    2019-06-10  21167386  
0          CONTRAST PLEATED DRESS       999900    2019-06-10  21165031  
0                    RUSTIC DRESS       699900    2019-06-10  21037716  
0            SATIN CAMISOLE DRESS       699900    2019-06-10  2103771

            category discounted_price  \
0  knitwear > basics           449900   
0  knitwear > basics           299900   
0  knitwear > basics           299900   
0  knitwear > basics           479900   
0  knitwear > basics           599900   

                                               image  \
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   

                         name origin_price scraping_date       sku  
0  BASIC SHORT SLEEVE SWEATER       449900    2019-06-10  13319882  
0               BASIC SWEATER       299900    2019-06-10  13319902  
0               BASIC SWEATER       299900    2019-06-10  13319913  
0  BASIC SWEATER WITH BUTTONS       479900    2019-06-10  13274260  
0    OVERSIZED V-NECK SWEATER       599900    2019-06-10   8301512  
sh

                          category discounted_price  \
0  t-shirts > graphics_and_slogans           379900   
0  t-shirts > graphics_and_slogans           379900   
0  t-shirts > graphics_and_slogans           299900   
0  t-shirts > graphics_and_slogans           229900   
0  t-shirts > graphics_and_slogans           189900   

                                               image  \
0  https://static.zara.net/photos///2019/I/0/1/p/...   
0  https://static.zara.net/photos///2019/I/0/1/p/...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   

                             name origin_price scraping_date       sku  
0  THE LION KING © DISNEY T-SHIRT       379900    2019-06-10  14172868  
0        © DISNEY’S DUMBO T-SHIRT       379900    2019-06-10  14172867  
0        T-SHIRT WITH FRONT PRINT       299900    2019-06-10  15874390  
0        T-SHIRT WITH FRONT PRINT       

              category discounted_price  \
0  trousers > wide leg           549900   
0  trousers > wide leg           549900   
0  trousers > wide leg           549900   
0  trousers > wide leg           549900   
0  trousers > wide leg           999900   

                                               image  \
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/photos///2019/V/0/1/p/...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   

                        name origin_price scraping_date       sku  
0  FLOWING WIDE-LEG TROUSERS       549900    2019-06-10  20782116  
0  FLOWING WIDE-LEG TROUSERS       549900    2019-06-10  21037834  
0  FLOWING WIDE-LEG TROUSERS       549900    2019-06-10  21037832  
0           PLEATED TROUSERS       549900    2019-06-10  21164967  
0       WIDE DRAPEY TROUSERS       999900    2019-06-10  1469430

...

               category discounted_price  \
0  knitwear > cardigans           699900   
0  knitwear > cardigans           899900   
0  knitwear > cardigans           479900   
0  knitwear > cardigans           899900   
0  knitwear > cardigans          1199900   

                                               image  \
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   

                                     name origin_price scraping_date      sku  
0  MULTICOLOURED VERTICAL STRIPE CARDIGAN       699900    2019-06-10  7886345  
0                        STRIPED CARDIGAN       899900    2019-06-10  8190581  
0                   COLOUR BLOCK CARDIGAN       479900    2019-06-10  7886228  
0                       JACQUARD CARDIGAN       899900    2019-06-10  7887827 

              category discounted_price  \
0  jeans > regular fit           599900   
0  jeans > regular fit           599900   
0  jeans > regular fit           599900   
0  jeans > regular fit           599900   
0  jeans > regular fit           599900   

                                               image  \
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/stdstatic/1.110.0-b.83...   

                            name origin_price scraping_date       sku  
0         BASIC ESSENTIALS JEANS       599900    2019-06-10  21005740  
0         BASIC ESSENTIALS JEANS       599900    2019-06-10   8389716  
0         BASIC ESSENTIALS JEANS       599900    2019-06-10   9033329  
0         BASIC ESSENTIALS JEANS       599900    2019-06-10   8412205  
0  BASIC STRAIGHT SLIM FIT JEANS       59990

           category discounted_price  \
0  shirts > printed           599900   
0  shirts > printed           599900   
0  shirts > printed           699900   
0  shirts > printed           599900   
0  shirts > printed           599900   

                                               image                  name  \
0  https://static.zara.net/photos///2019/I/0/2/p/...  ABSTRACT PRINT SHIRT   
0  https://static.zara.net/photos///2019/I/0/2/p/...  ABSTRACT PRINT SHIRT   
0  https://static.zara.net/photos///2019/I/0/2/p/...      LEAF PRINT SHIRT   
0  https://static.zara.net/stdstatic/1.110.0-b.83...    FLORAL PRINT SHIRT   
0  https://static.zara.net/stdstatic/1.110.0-b.83...    ISLAND PRINT SHIRT   

  origin_price scraping_date       sku  
0       599900    2019-06-10  16010902  
0       599900    2019-06-10  15194445  
0       699900    2019-06-10  14944306  
0       599900    2019-06-10   8679672  
0       599900    2019-06-10  14864913  
shirts > denim
Connected successfully!!!
   

       category discounted_price  \
0  polo > plain           479900   
0  polo > plain           479900   
0  polo > plain           479900   
0  polo > plain           479900   
0  polo > plain           479900   

                                               image  \
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/V/0/2/p/...   
0  https://static.zara.net/photos///2019/I/0/2/p/...   

                              name origin_price scraping_date       sku  
0  POLO SHIRT WITH CONTRAST COLLAR       479900    2019-06-10  11735574  
0  POLO SHIRT WITH CONTRAST COLLAR       479900    2019-06-10  11735577  
0  POLO SHIRT WITH CONTRAST COLLAR       479900    2019-06-10  11735576  
0  POLO SHIRT WITH CONTRAST COLLAR       479900    2019-06-10  11735575  
0     CONTRASTING PIQUÉ POLO SHIRT       479900    2019-06-10  13844880  
po

          category discounted_price  \
0  shoes > loafers          1399900   
0  shoes > loafers          1399900   
0  shoes > loafers   1399900 999900   
0  shoes > loafers          1399900   
0  shoes > loafers          1699900   

                                               image  \
0  https://static.zara.net/photos///2019/V/1/2/p/...   
0  https://static.zara.net/photos///2019/V/1/2/p/...   
0  https://static.zara.net/photos///2019/V/1/2/p/...   
0  https://static.zara.net/photos///2019/V/1/2/p/...   
0  https://static.zara.net/photos///2019/I/1/2/p/...   

                         name    origin_price scraping_date       sku  
0  BLACK LEATHER MULE LOAFERS         1399900    2019-06-10  18861032  
0        SMART TASSEL LOAFERS         1399900    2019-06-10  10404856  
0        SMART TASSEL LOAFERS  1399900 999900    2019-06-10   7904596  
0     CLASSIC LEATHER LOAFERS         1399900    2019-06-10   8258077  
0             LEATHER LOAFERS         1699900    2019-06-10  1333875

In [108]:
# Scraping the page
def scrape_detail_page2(url,category,collection):
    
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.zara
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)
    result = pd.DataFrame()
    item = driver.find_elements_by_css_selector('.item._item')
    
    for x in range(0,len(item)):
        get_link = item[x]
        print("Go to detail page: " + get_link.get_attribute("href"))

        # Save the window opener (current window, do not mistaken with tab... not the same)
        main_window = driver.current_window_handle

        # Open the link in a new tab by sending key strokes on the element
        # Use: Keys.CONTROL + Keys.SHIFT + Keys.RETURN to open tab on top of the stack 
        get_link.send_keys(Keys.CONTROL + Keys.RETURN)

        # Switch tab to the new tab, which we will assume is the next one on the right
        driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.TAB)
        time.sleep(5)

       # Get windows list and put focus on new window (which is on the 1st index in the list)
        while(True):
            try:
                windows = driver.window_handles
                driver.switch_to.window(windows[1])
                break
            except:
                time.sleep(5)

        # Go to detail page
        result = result.append(scrape_detail_page3(driver,category,collection))
        # Close current tab
        driver.close()

        # Put focus on current window which will be the window opener
        driver.switch_to_window(main_window)


    driver.close()
    return result

In [109]:
def scrape_detail_page3(driver,category,collection):
    path = driver.find_elements_by_xpath('/html/body/div[2]/section/section/div[1]/div/div[2]/div[2]/div/div/ul/li')
    result = pd.DataFrame()
    for x in range(len(path)):
        try:
            price = re.sub("\,","",re.sub(" IDR","",path[x].find_element_by_css_selector('.price._product-price').text))
            name = path[x].find_element_by_css_selector('.title').text
            sku = path[x].find_elements_by_css_selector('img')[0].get_attribute('data-id')
            image = path[x].find_elements_by_css_selector('img')[0].get_attribute('src')
            while(image == 'https://static.zara.net/stdstatic/1.108.1-b.2/images/transparent-background.png'):
                image = path[x].find_elements_by_css_selector('img')[0].get_attribute('src')
                time.sleep(2)
            # Return result
            date = datetime.datetime.now().strftime("%Y-%m-%d")
            new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':price, 'discounted_price':price, 'image':image,'category':category,'scraping_date':date}])
            result = result.append(new_result)
            
            # save to mongodb
            collection.insert_many(new_result.to_dict('records'))
        except:
            PrintException()    
    return result

In [ ]:
# MAIN
# only for zara_pria ['basics > suits','suits > formal','suits > casual','tracksuits']
zara_category = pd.read_csv('C:\\Users\\tamu\\SharePoint\\Product Development - Documents\\Merchandising Analytics\\Analysis - QBR Jul19\\Data\\scraping\\zara_category.csv')
zara_category = zara_category[zara_category['category'].isin(['basics > suits','suits > formal','suits > casual','tracksuits'])]
result = pd.DataFrame()
for x in range(2,4):
    category = zara_category.iloc[x]
    print(category['category'])
    result = scrape_detail_page2(category['url'],category['category'],category['collection'])
    print(result.head())